In [6]:
import geopandas as gpd    
import fiona
import pandas as pd
from datetime import datetime
import loguru
import sys
import os
import argparse
from pathlib import Path
from lxml import etree
from shapely.geometry import Point

## Parse kml file

In [4]:
file_name = "/media/manecao/HD/Desktop/projetos-2025/ICMBIO/project/exemplo/Ocurrencia12.kml"

In [7]:

# Parse the KML file
tree = etree.parse(file_name)
root = tree.getroot()

# Define namespaces
ns = {'kml': 'http://www.opengis.net/kml/2.2'}

# Extract placemarks with schema1 data
data = []
for placemark in root.findall('.//kml:Placemark', ns):
    schema_data = placemark.find('.//kml:SchemaData[@schemaUrl="#schema1"]', ns)
    if schema_data is not None:
        # Get coordinates
        coords_text = placemark.find('.//kml:coordinates', ns).text.strip()
        lon, lat, alt = map(float, coords_text.split(','))
        
        # Get name
        name = placemark.find('.//kml:name', ns).text
        
        # Get timestamp if available
        timestamp_elem = placemark.find('.//kml:when', ns)
        timestamp = timestamp_elem.text if timestamp_elem is not None else None
        
        # Extract all SimpleData fields
        record = {'name': name,
                  'geometry': Point(lon, lat),
                  'elevation':alt}
        for simple_data in schema_data.findall('.//kml:SimpleData', ns):
            field_name = simple_data.get('name')
            field_value = simple_data.text
            record[field_name] = field_value
        
        data.append(record)

# Create GeoDataFrame
gdf = gpd.GeoDataFrame(data, crs='EPSG:4326').rename(str.lower, axis='columns')
print(gdf)

              name                     geometry  ...    description        descricao
0      Placemark 1  POINT (-48.51494 -27.46828)  ...            NaN              NaN
1  Placemark 1 (1)  POINT (-48.50344 -27.46536)  ...  Qlqr coisa aq  mais qlqr coisa
2       Placemark2   POINT (-48.51101 -27.4637)  ...            NaN              NaN

[3 rows x 14 columns]


In [8]:
gdf

,name,geometry,elevation,nome cientifico,nivel prioridade,risco da invasao,estagio invasao,grau dispersao,num_individuos,zone,metodo,area degradada,description,descricao
0,Placemark 1,POINT (-48.51494 -27.46828),4.821764,Pinus Sp,3,2 - Moderado,3 - Invasora,B - Pontual,20,3 - Papaquara Sul,mecanico,nao,NaN,NaN
1,Placemark 1 (1),POINT (-48.50344 -27.46536),3.629106,Pinus Sp,4,2 - Moderado,2 - Estabelecida,C - Dispersa,20,4 - Rancho Ratones,NaN,degradacao por invasao,Qlqr coisa aq,mais qlqr coisa
2,Placemark2,POINT (-48.51101 -27.4637),4.196405,Heptapleurum Actinophyllum,3,3 - Baixo,0 - Contida,A - Localizada,0,4 - Rancho Ratones,quimico,degradacao por invasao,NaN,NaN


In [181]:
gdf['adultos removidos']
pd.to_numeric(gdf['adultos removidos'], errors="coerce" ).astype("Int64").info()

<class 'pandas.core.series.Series'>
RangeIndex: 3 entries, 0 to 2
Series name: adultos removidos
Non-Null Count  Dtype
--------------  -----
1 non-null      Int64
dtypes: Int64(1)
memory usage: 159.0 bytes


In [9]:
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   name              3 non-null      object  
 1   geometry          3 non-null      geometry
 2   elevation         3 non-null      float64 
 3   nome cientifico   3 non-null      object  
 4   nivel prioridade  3 non-null      object  
 5   risco da invasao  3 non-null      object  
 6   estagio invasao   3 non-null      object  
 7   grau dispersao    3 non-null      object  
 8   num_individuos    3 non-null      object  
 9   zone              3 non-null      object  
 10  metodo            2 non-null      object  
 11  area degradada    3 non-null      object  
 12  description       1 non-null      object  
 13  descricao         1 non-null      object  
dtypes: float64(1), geometry(1), object(12)
memory usage: 468.0+ bytes


In [16]:
gdf['x'] = gdf.geometry.x
gdf['y'] = gdf.geometry.y

## export the gdf without the geometry
gdf.drop(columns='geometry').to_csv('test.csv', index=False)

In [ ]:
## read csv to gdf

## add main 
## add argparse for the name of the file
## add argparse for the folder 

file_name ="test.csv"
gdf2 = gpd.GeoDataFrame(
    pd.read_csv(file_name),
    geometry=gpd.points_from_xy(pd.read_csv(file_name).x, pd.read_csv(file_name).y),
    crs='EPSG:4326'
)

In [10]:
import pyogrio

pyogrio.list_drivers()

{'PCIDSK': 'rw',
 'PDS4': 'rw',
 'VICAR': 'rw',
 'PDF': 'rw',
 'MBTiles': 'rw',
 'EEDA': 'r',
 'OGCAPI': 'r',
 'ESRI Shapefile': 'rw',
 'MapInfo File': 'rw',
 'UK .NTF': 'r',
 'LVBAG': 'r',
 'OGR_SDTS': 'r',
 'S57': 'rw',
 'DGN': 'rw',
 'OGR_VRT': 'r',
 'Memory': 'rw',
 'CSV': 'rw',
 'GML': 'rw',
 'GPX': 'rw',
 'KML': 'rw',
 'GeoJSON': 'rw',
 'GeoJSONSeq': 'rw',
 'ESRIJSON': 'r',
 'TopoJSON': 'r',
 'OGR_GMT': 'rw',
 'GPKG': 'rw',
 'SQLite': 'rw',
 'WAsP': 'rw',
 'OpenFileGDB': 'rw',
 'DXF': 'rw',
 'FlatGeobuf': 'rw',
 'Geoconcept': 'rw',
 'GeoRSS': 'rw',
 'VFK': 'r',
 'PGDUMP': 'rw',
 'OSM': 'r',
 'GPSBabel': 'rw',
 'OGR_PDS': 'r',
 'WFS': 'r',
 'OAPIF': 'r',
 'EDIGEO': 'r',
 'SVG': 'r',
 'Idrisi': 'r',
 'ODS': 'rw',
 'XLSX': 'rw',
 'Elasticsearch': 'rw',
 'Carto': 'rw',
 'AmigoCloud': 'rw',
 'SXF': 'r',
 'Selafin': 'rw',
 'JML': 'rw',
 'PLSCENES': 'r',
 'CSW': 'r',
 'VDV': 'rw',
 'MVT': 'rw',
 'NGW': 'rw',
 'MapML': 'rw',
 'GTFS': 'r',
 'PMTiles': 'rw',
 'JSONFG': 'rw',
 'MiraMonVecto

In [10]:
for col in gdf.columns:
    print(col.lower())

name
geometry
elevation
tipo de acao
zona
especie
status remocao
individuos
plantulas removidas
jovens removidos
metodo de controle
controle mecanico
quimico aplicado?
quimico concentracao
quimico litros
inicio acao hh:mm:ss
fim acao hh:mm:ss
num. manejadores
num equipe
custo
description
adultos removidos
principio ativo
comentario


In [8]:
gdf.columns

Index(['name', 'geometry', 'elevation', 'nome cientifico', 'nivel prioridade',
       'risco da invasao', 'estagio invasao', 'grau dispersao',
       'num_individuos', 'zone', 'metodo', 'area degradada'],
      dtype='object')

In [11]:
def apply_schema(gdf, schema):
    for col, dtype in schema.items():
        if col not in gdf.columns:
            continue  # safely skip missing columns
        if dtype == "datetime":
            print(col)
            gdf[col] = pd.to_datetime(gdf[col], errors="coerce")

        elif dtype == "int":
            gdf[col] = pd.to_numeric(gdf[col], errors="coerce").astype("Int64")

        elif dtype == "float":
            gdf[col] = pd.to_numeric(gdf[col], errors="coerce")

        elif dtype == "bool":
            gdf[col] = gdf[col].apply(lambda x:bool(x))

    return gdf

def apply_cleaning(gdf, list_cols=list):
    """
    Split the text input referenced by "-". Return the number which represents the first
    item of the list.
    """
    def clean_txt(txt):
        s = txt.split('-')
        if len(s)>0:
            return s[0]
        else:
            return s
        
    ## loop through the list
    for col in list_cols:
        if col in gdf.columns:
            gdf[col]= gdf[col].apply(lambda x: clean_txt(x))

    return gdf

In [12]:
schema =  {
    "name": "str",
    "elevation": "float",

    "tipo de acao": "str",
    "zona": "int",
    "especie": "str",
    "status remocao": "str",

    "individuos": "int",
    "plantulas removidas": "int",
    "jovens removidos": "int",
    "adultos removidos":"int",


    "metodo de controle": "str",
    "controle mecanico": "str",
    "quimico aplicado?": "bool",
    "principio ativo":"int",
    "quimico concentracao": "float",
    "quimico litros": "float",

    "inicio acao hh:mm:ss": "datetime",
    "fim acao hh:mm:ss": "datetime",
    "num. manejadores": "int",
    "num equipe": "int",
    "custo": "float",
    "description":"str",
    "comentario":"str"
  }

In [21]:
list_cols = ['risco da invasao','estagio invasao','grau dispersao','zona']
ggdf = apply_cleaning(gdf, list_cols)
ggdf = apply_schema(ggdf, schema)

inicio acao hh:mm:ss
fim acao hh:mm:ss


/tmp/ipykernel_180248/2712413655.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  gdf[col] = pd.to_datetime(gdf[col], errors="coerce")
/tmp/ipykernel_180248/2712413655.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  gdf[col] = pd.to_datetime(gdf[col], errors="coerce")


In [19]:
ggdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   name                  3 non-null      object  
 1   geometry              3 non-null      geometry
 2   elevation             3 non-null      float64 
 3   tipo de acao          3 non-null      object  
 4   zona                  3 non-null      object  
 5   especie               3 non-null      object  
 6   status remocao        3 non-null      object  
 7   individuos            3 non-null      object  
 8   plantulas removidas   3 non-null      object  
 9   jovens removidos      3 non-null      object  
 10  metodo de controle    3 non-null      object  
 11  controle mecanico     3 non-null      object  
 12  quimico aplicado?     3 non-null      object  
 13  quimico concentracao  3 non-null      object  
 14  quimico litros        3 non-null      object  
 15  in

In [22]:
gdf

,name,geometry,elevation,tipo de acao,zona,especie,status remocao,individuos,plantulas removidas,jovens removidos,...,quimico litros,inicio acao hh:mm:ss,fim acao hh:mm:ss,num. manejadores,num equipe,custo,description,adultos removidos,principio ativo,comentario
0,1,POINT (-48.50045 -27.46048),2.838002,Primeira Acao,1,Pinus Sp,Parcial,25,0,0,...,0,2025-11-22 13:45:00,2025-11-22 17:30:00,2,3,75.00,NaN,<NA>,<NA>,NaN
1,Placemark 1,POINT (-48.51527 -27.46817),4.019994,Resposta Rapida,2,Pinus Sp,Total,20,0,0,...,0,2025-11-22 14:00:00,2025-11-22 16:00:00,5,5,50.00,NaN,<NA>,<NA>,NaN
2,Placemark 1 (1),POINT (-48.50476 -27.46593),4.421778,Repasse,1,Syzygium Cumini,Total,25,15,2,...,5,2025-11-22 13:45:00,2025-11-22 17:30:00,3,4,197.45,"Descricao in description, let's see",5,<NA>,qualquer comentário final sobre a operação


In [61]:
gdf.shape

(2, 20)

In [94]:
ggdf.shape

(2, 20)

In [100]:
ggdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   name                  2 non-null      object  
 1   geometry              2 non-null      geometry
 2   elevation             2 non-null      float64 
 3   Tipo de Acao          2 non-null      object  
 4   Zona                  2 non-null      object  
 5   Especie               2 non-null      object  
 6   Status Remocao        2 non-null      object  
 7   Individuos            2 non-null      object  
 8   Plantulas Removidas   2 non-null      object  
 9   Jovens Removidos      2 non-null      object  
 10  Metodo de Controle    2 non-null      object  
 11  Controle Mecanico     2 non-null      object  
 12  Quimico Aplicado?     2 non-null      object  
 13  Quimico Concentracao  2 non-null      object  
 14  Quimico Litros        2 non-null      object  
 15  In

In [102]:
gdf["Num equipe"] 

0    3
1    5
Name: Num equipe, dtype: object

In [92]:
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   name                  2 non-null      object  
 1   geometry              2 non-null      geometry
 2   elevation             2 non-null      float64 
 3   Tipo de Acao          2 non-null      object  
 4   Zona                  2 non-null      object  
 5   Especie               2 non-null      object  
 6   Status Remocao        2 non-null      object  
 7   Individuos            2 non-null      object  
 8   Plantulas Removidas   2 non-null      object  
 9   Jovens Removidos      2 non-null      object  
 10  Metodo de Controle    2 non-null      object  
 11  Controle Mecanico     2 non-null      object  
 12  Quimico Aplicado?     2 non-null      object  
 13  Quimico Concentracao  2 non-null      object  
 14  Quimico Litros        2 non-null      object  
 15  In

In [97]:
ggdf

,name,geometry,elevation,Tipo de Acao,Zona,Especie,Status Remocao,Individuos,Plantulas Removidas,Jovens Removidos,Metodo de Controle,Controle Mecanico,Quimico Aplicado?,Quimico Concentracao,Quimico Litros,Inicio Acao HH:mm:ss,Fim Acao HH:mm:ss,Num. Manejadores,Num equipe,Custo
0,1,POINT (-48.50045 -27.46048),2.838002,Primeira Acao,1 - Margem Brincas,Pinus Sp,Parcial,25,0,0,Combinado,Corte Raso,0,0,0,13:45:00,17:30:00,2,3,75
1,Placemark 1,POINT (-48.51527 -27.46817),4.019994,Resposta Rapida,2 - Papaquara Norte,Pinus Sp,Total,20,0,0,Mecanico,Arranquio,0,0,0,14:00:00,16:00:00,5,5,50


# 2. Casts the gpkg into the DB format

In [240]:
gdf_ps = gpd.read_file("output/processed_data/ocurrencia12_ps.gpkg")

In [242]:
gdf_ps

,name,elevation,especie,nivel_prioridade,risco_invasao,estagio_invasao,grau_dispersao,individuos,zona,area_degradada,description,comentario,date,time,geometry
0,Placemark 1,4.821764,Pinus Sp,3,2,3,B,20,3,NaN,None,None,2025-11-21,11:15:47,POINT (-48.51494 -27.46828)
1,Placemark 1 (1),3.629106,Pinus Sp,4,2,2,C,20,4,NaN,Qlqr coisa aq,mais qlqr coisa,2025-11-22,18:14:43,POINT (-48.50344 -27.46536)
2,Placemark2,4.196405,Heptapleurum Actinophyllum,3,3,0,A,0,4,NaN,None,None,2025-11-21,11:17:24,POINT (-48.51101 -27.4637)


In [143]:
map_gpd_db_ocorrencia = {
    'id':'id',
    'elevation':"elevation",
    'date':"date",
    "time":"time",
    'nome cientifico':"especie",
    'nivel prioridade':"nivel_prioridade",
    'risco da invasao':"risco_invasao",
    'estagio invasao':"estagio_invasao",
    'grau dispersao':"grau_dispersao",
    'num_individuos':'individuos',
    'zone':"zona",
    'area degradada':"area_degradada",
    'description':"description",
    'descricao':"comentario",
    'geometry':"geom"
}

map_gpd_db_manejo = {
    "id": "id",
    "elevation": "elevation",
    "date": "date",
    "time": "time",

    "tipo de acao": "tipo_acao",
    "zona": "zona",
    "especie": "especie",
    "status remocao": "status_remocao",

    "individuos": "individuos",
    "plantulas removidas": "plantulas_rev",
    "jovens removidos": "jovens_rev",
    "adultos removidos": "adultos_rev",

    "metodo de controle": "metodo_controle",
    "controle mecanico": "mec_controle",

    "principio ativo": "principio_ativo",
    "quimico concentracao": "quimic_concentr",
    "quimico litros": "quimic_l",

    "inicio acao hh:mm:ss": "inicio",
    "fim acao hh:mm:ss": "fim",

    "num. manejadores": "num_manej",
    "num equipe": "num_equipe",

    "custo": "custo",

    "geometry": "geom",
    'description':"description",
    'comentario':"comentario",
}

In [152]:
# Rename columns based on the mapping
ggdf = gdf_ps.rename(columns=map_gpd_db_manejo)

# Columns expected by the DB (values of the mapping dict)
cols_db_manejo = set(map_gpd_db_manejo.values())

# Identify columns not present in the DB table
cols_to_drop = ggdf.columns.difference(cols_db_manejo)

print(f"Columns that don't match the database: {list(cols_to_drop)}")

# Drop unnecessary columns
print(f"Before Drop:{ggdf.shape}")
ggdf = ggdf.drop(columns=list(cols_to_drop),axis='columns')
print(f"After Drop: {ggdf.shape}")


Columns that don't match the database: ['date_og', 'name', 'quimico aplicado?']
Before Drop:(3, 28)
Afer Drop: (3, 25)


In [148]:
if not isinstance(cols_to_drop,list):
    ## try see if is an index or columns
    try:
        isinstance(cols_to_drop, pd.Index):
            cols_to_drop = cols_to_drop.tolist()
    
    exception:
        

False

In [150]:
isinstance(cols_to_drop, pd.Index)

True

In [151]:
cols_to_drop.tolist()

['date_og', 'name', 'quimico aplicado?']

### Test

In [207]:
gdf= gpd.read_file("output/processed_data/Manejo1_ps.gpkg")

/home/manecomaneca/venv/lighthouse/.venv/lib/python3.13/site-packages/pyogrio/raw.py:198: RuntimeWarning: Non-conformant content for record 1 in column date_og, 2025-11-21T11:18:32.000+01:00, successfully parsed
  return ogr_read(


In [211]:
gdf= gpd.read_file("output/processed_data/Manejo1_ps.gpkg")

In [214]:
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 25 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   name             3 non-null      object  
 1   elevation        3 non-null      float64 
 2   tipo_acao        3 non-null      object  
 3   zona             3 non-null      int64   
 4   especie          3 non-null      object  
 5   status_remocao   3 non-null      object  
 6   individuos       3 non-null      int64   
 7   plantulas_rev    3 non-null      int64   
 8   jovens_rev       3 non-null      int64   
 9   metodo_controle  3 non-null      object  
 10  mec_controle     3 non-null      object  
 11  quimic_concentr  3 non-null      float64 
 12  quimic_l         3 non-null      int64   
 13  inicio           3 non-null      object  
 14  fim              3 non-null      object  
 15  num_manej        3 non-null      int64   
 16  num_equipe       3 non-null      int64  

## CONNECT DB


In [218]:

import psycopg2

# Connect to your database
conn = psycopg2.connect(
    host="172.30.32.1",        # or your server address
    database="manejo",
    user="postgres",
    password="m@n3c0!",
    port="5432"              # default PostgreSQL port
)

# Create a cursor
cur = conn.cursor()

# Execute a query
# cur.execute("SELECT * FROM ocorrencia LIMIT 5;")

table = "manejo"

## query the columns of a given database
cur.execute(
    """ 
SELECT column_name, data_type, is_nullable
FROM information_schema.columns
WHERE table_name = 'manejo' 
AND table_schema = 'public'
ORDER BY ordinal_position;
"""
)


# Fetch results
rows = cur.fetchall()
for row in rows:
    print(row)
    
dict_out = {}
dict_out['cols'] = [row[0] for row in rows]
dict_out['dtypes'] = [row[1] for row in rows] 
dict_out['nullable'] = [row[2] for row in rows] 

# Close connections
cur.close()
conn.close()

('id', 'integer', 'NO')
('name', 'character varying', 'YES')
('elevation', 'numeric', 'YES')
('date', 'date', 'NO')
('time', 'time without time zone', 'NO')
('tipo_acao', 'character varying', 'YES')
('zona', 'integer', 'YES')
('especie', 'character varying', 'NO')
('status_remocao', 'character varying', 'YES')
('individuos', 'integer', 'NO')
('plantulas_rev', 'integer', 'YES')
('jovens_rev', 'integer', 'YES')
('adultos_rev', 'integer', 'YES')
('metodo_controle', 'character varying', 'YES')
('mec_controle', 'character varying', 'YES')
('principio_ativo', 'character varying', 'YES')
('quimic_concentr', 'numeric', 'YES')
('quimic_l', 'numeric', 'YES')
('inicio', 'time without time zone', 'YES')
('fim', 'time without time zone', 'YES')
('num_manej', 'integer', 'YES')
('num_equipe', 'integer', 'YES')
('custo', 'numeric', 'YES')
('geom', 'USER-DEFINED', 'NO')
('comentario', 'character varying', 'YES')
('description', 'character varying', 'YES')
('created_at', 'timestamp without time zone', '

In [228]:
dict(zip(dict_out['cols'],dict_out['dtypes']))

{'id': 'integer',
 'name': 'character varying',
 'elevation': 'numeric',
 'date': 'date',
 'time': 'time without time zone',
 'tipo_acao': 'character varying',
 'zona': 'integer',
 'especie': 'character varying',
 'status_remocao': 'character varying',
 'individuos': 'integer',
 'plantulas_rev': 'integer',
 'jovens_rev': 'integer',
 'adultos_rev': 'integer',
 'metodo_controle': 'character varying',
 'mec_controle': 'character varying',
 'principio_ativo': 'character varying',
 'quimic_concentr': 'numeric',
 'quimic_l': 'numeric',
 'inicio': 'time without time zone',
 'fim': 'time without time zone',
 'num_manej': 'integer',
 'num_equipe': 'integer',
 'custo': 'numeric',
 'geom': 'USER-DEFINED',
 'comentario': 'character varying',
 'description': 'character varying',
 'created_at': 'timestamp without time zone',
 'updated_at': 'timestamp without time zone'}

In [ ]:
cols_db = [row[0] for row in rows]
col_dtype_db = [row[1] for row in rows]
nullable_col_db = [row[2] for row in rows]

new_dict = {}
for i, col in enumerate(cols_db):
    new_dict[col] = col_dtype_db[i]

In [209]:
new_dict

{'id': 'integer',
 'name': 'character varying',
 'elevation': 'numeric',
 'date': 'date',
 'time': 'time without time zone',
 'tipo_acao': 'character varying',
 'zona': 'integer',
 'especie': 'character varying',
 'status_remocao': 'character varying',
 'individuos': 'integer',
 'plantulas_rev': 'integer',
 'jovens_rev': 'integer',
 'adultos_rev': 'integer',
 'metodo_controle': 'character varying',
 'mec_controle': 'character varying',
 'principio_ativo': 'character varying',
 'quimic_concentr': 'numeric',
 'quimic_l': 'numeric',
 'inicio': 'time without time zone',
 'fim': 'time without time zone',
 'num_manej': 'integer',
 'num_equipe': 'integer',
 'custo': 'numeric',
 'geom': 'USER-DEFINED',
 'comentario': 'character varying',
 'description': 'character varying',
 'created_at': 'timestamp without time zone',
 'updated_at': 'timestamp without time zone'}

## Casting before the db

In [201]:
import pandas as pd
import geopandas as gpd
from typing import Dict
import numpy as np
from loguru import logger


def get_pandas_dtype_map() -> Dict[str, str]:
    """
    Map PostgreSQL data types to pandas dtypes.
    
    Returns:
        Dictionary mapping PostgreSQL types to pandas types
    """
    return {
        'integer': 'Int64',  # Nullable integer
        'bigint': 'Int64',
        'smallint': 'Int32',
        'numeric': 'float64',
        'real': 'float32',
        'double precision': 'float64',
        'character varying': 'string',
        'varchar': 'string',
        'text': 'string',
        'char': 'string',
        'character': 'string',
        'date': 'object',  # Will convert later
        'time without time zone': 'object',  # Will convert later
        'time with time zone': 'object',
        'timestamp without time zone': 'object',  # Will convert later
        'timestamp with time zone': 'object',
        'boolean': 'boolean',
        'bool': 'boolean',
        'USER-DEFINED': None,  # Skip geometry and custom types
    }


def cast_gdf_to_schema(gdf: gpd.GeoDataFrame, pg_schema: Dict[str, str]) -> gpd.GeoDataFrame:
    """
    Cast GeoDataFrame columns to match PostgreSQL schema.
    
    Args:
        gdf: GeoDataFrame to cast
        pg_schema: Dictionary mapping column names to PostgreSQL types
        
    Returns:
        GeoDataFrame with properly typed columns
    """
    dtype_map = get_pandas_dtype_map()
    gdf_copy = gdf.copy()
    
    for col, pg_type in pg_schema.items():
        # Skip if column doesn't exist in GeoDataFrame
        if col not in gdf_copy.columns:
            logger.debug(f"Column '{col}' not found in GeoDataFrame, skipping")
            continue
        
        # Skip geometry column
        if col == 'geom' or col == 'geometry':
            continue
        
        # Skip auto-generated columns
        if col in ['id', 'created_at', 'updated_at']:
            continue
        
        # Get corresponding pandas dtype
        pandas_dtype = dtype_map.get(pg_type)
        
        if pandas_dtype is None:
            logger.debug(f"Skipping column '{col}' with type '{pg_type}'")
            continue
        
        try:
            # Handle date/time types specially
            if pg_type == 'date':
                gdf_copy[col] = pd.to_datetime(gdf_copy[col], errors='coerce').dt.date
            elif pg_type in ['time without time zone', 'time with time zone']:
                gdf_copy[col] = pd.to_datetime(gdf_copy[col], format='%H:%M:%S', errors='coerce').dt.time
            elif pg_type in ['timestamp without time zone', 'timestamp with time zone']:
                gdf_copy[col] = pd.to_datetime(gdf_copy[col], errors='coerce')
            else:
                # Cast to the appropriate dtype
                if pandas_dtype == 'Int64':
                    # Handle nullable integers
                    gdf_copy[col] = pd.to_numeric(gdf_copy[col], errors='coerce').astype('Int64')
                elif pandas_dtype in ['float64', 'float32']:
                    gdf_copy[col] = pd.to_numeric(gdf_copy[col], errors='coerce')
                elif pandas_dtype == 'string':
                    gdf_copy[col] = gdf_copy[col].astype('string')
                elif pandas_dtype == 'boolean':
                    gdf_copy[col] = gdf_copy[col].astype('boolean')
                else:
                    gdf_copy[col] = gdf_copy[col].astype(pandas_dtype)
            
            logger.debug(f"Converted column '{col}' from {gdf[col].dtype} to {pandas_dtype} ({pg_type})")
            
        except Exception as e:
            logger.warning(f"Failed to convert column '{col}' to {pandas_dtype}: {e}")
    
    return gdf_copy


def validate_schema_match(gdf: gpd.GeoDataFrame, pg_schema: Dict[str, str]) -> Dict[str, str]:
    """
    Validate which columns from schema are present in GeoDataFrame.
    
    Args:
        gdf: GeoDataFrame to validate
        pg_schema: PostgreSQL schema dictionary
        
    Returns:
        Dictionary of validation results
    """
    results = {
        'present': [],
        'missing': [],
        'extra': []
    }
    
    # Exclude auto-generated columns
    auto_columns = ['id', 'created_at', 'updated_at']
    schema_cols = set(pg_schema.keys()) - set(auto_columns) - {'geom'}
    gdf_cols = set(gdf.columns) - {'geometry'}
    
    results['present'] = list(schema_cols & gdf_cols)
    results['missing'] = list(schema_cols - gdf_cols)
    results['extra'] = list(gdf_cols - schema_cols)
    
    logger.info(f"Schema validation: {len(results['present'])} matched, "
                f"{len(results['missing'])} missing, {len(results['extra'])} extra")
    
    if results['missing']:
        logger.warning(f"Missing columns in GeoDataFrame: {results['missing']}")
    if results['extra']:
        logger.info(f"Extra columns in GeoDataFrame: {results['extra']}")
    
    return results

In [ ]:
gdf_ps.

In [215]:
validate_schema = validate_schema_match(gdf, new_dict)

ggdf = cast_gdf_to_schema(gdf, new_dict)

2025-11-23 07:39:34.313 | INFO     | __main__:validate_schema_match:129 - Schema validation: 24 matched, 0 missing, 0 extra


{'present': ['principio_ativo',
  'metodo_controle',
  'elevation',
  'quimic_concentr',
  'num_manej',
  'date',
  'fim',
  'especie',
  'mec_controle',
  'comentario',
  'description',
  'plantulas_rev',
  'name',
  'tipo_acao',
  'jovens_rev',
  'quimic_l',
  'time',
  'zona',
  'adultos_rev',
  'status_remocao',
  'individuos',
  'num_equipe',
  'custo',
  'inicio'],
 'missing': [],
 'extra': []}

In [216]:
ggdf = cast_gdf_to_schema(gdf, new_dict)

2025-11-23 07:42:51.755 | DEBUG    | __main__:cast_gdf_to_schema:55 - Column 'id' not found in GeoDataFrame, skipping
2025-11-23 07:42:51.760 | DEBUG    | __main__:cast_gdf_to_schema:95 - Converted column 'name' from object to string (character varying)
2025-11-23 07:42:51.774 | DEBUG    | __main__:cast_gdf_to_schema:95 - Converted column 'elevation' from float64 to float64 (numeric)
2025-11-23 07:42:51.805 | DEBUG    | __main__:cast_gdf_to_schema:95 - Converted column 'date' from object to object (date)
2025-11-23 07:42:51.809 | DEBUG    | __main__:cast_gdf_to_schema:95 - Converted column 'time' from object to object (time without time zone)
2025-11-23 07:42:51.827 | DEBUG    | __main__:cast_gdf_to_schema:95 - Converted column 'tipo_acao' from object to string (character varying)
2025-11-23 07:42:51.833 | DEBUG    | __main__:cast_gdf_to_schema:95 - Converted column 'zona' from int64 to Int64 (integer)
2025-11-23 07:42:51.837 | DEBUG    | __main__:cast_gdf_to_schema:95 - Converted colu

In [ ]:

    # Load your GPKG
    gdf = gpd.read_file('output/processed_data/Manejo1_ps.gpkg')
    
    # Validate schema
    validation = validate_schema_match(gdf, manejo_schema)
    
    # Cast to proper types
    gdf_typed = cast_gdf_to_schema(gdf, manejo_schema)
    
    # Display info
    print("\nOriginal dtypes:")
    print(gdf.dtypes)
    print("\n" + "="*50)
    print("\nCasted dtypes:")
    print(gdf_typed.dtypes)
    
    # Show sample data
    print("\n" + "="*50)
    print("\nSample data:")
    print(gdf_typed.head())

## Standardize schema

In [232]:
import json
with open("schema.json", "r", encoding="utf-8") as f:
    schemas = json.load(f)

In [237]:

dict(zip([var['source_column'] for var in schemas['manejo']['mappings']],
         [var['data_type_source'] for var in schemas['manejo']['mappings']]))

{'name': 'str',
 'elevation': 'float',
 'date_og': 'datetime',
 'tipo de acao': 'str',
 'zona': 'int',
 'especie': 'str',
 'status remocao': 'str',
 'individuos': 'int',
 'plantulas removidas': 'int',
 'jovens removidos': 'int',
 'adultos removidos': 'int',
 'metodo de controle': 'str',
 'controle mecanico': 'str',
 'quimico aplicado?': 'bool',
 'principio ativo': 'int',
 'quimico concentracao': 'float',
 'quimico litros': 'float',
 'inicio acao hh:mm:ss': 'time',
 'fim acao hh:mm:ss': 'time',
 'num. manejadores': 'int',
 'num equipe': 'int',
 'custo': 'float',
 'description': 'str',
 'comentario': 'str'}

In [238]:
dict(zip([var['source_column'] for var in schemas['manejo']['mappings']],
         [var['db_column'] for var in schemas['manejo']['mappings']]))

{'name': 'name',
 'elevation': 'elevation',
 'date_og': 'time',
 'tipo de acao': 'tipo_acao',
 'zona': 'zona',
 'especie': 'especie',
 'status remocao': 'status_remocao',
 'individuos': 'individuos',
 'plantulas removidas': 'plantulas_rev',
 'jovens removidos': 'jovens_rev',
 'adultos removidos': 'adultos_rev',
 'metodo de controle': 'metodo_controle',
 'controle mecanico': 'mec_controle',
 'quimico aplicado?': 'quimico_aplicado',
 'principio ativo': 'principio_ativo',
 'quimico concentracao': 'quimic_concentr',
 'quimico litros': 'quimic_l',
 'inicio acao hh:mm:ss': 'inicio',
 'fim acao hh:mm:ss': 'fim',
 'num. manejadores': 'num_manej',
 'num equipe': 'num_equipe',
 'custo': 'custo',
 'description': 'description',
 'comentario': 'comentario'}

In [239]:
type(schema)

dict

## Testing functions from APP

In [1]:
from src.copypreprocess import parse_kml

In [2]:
file_name = "/media/manecao/HD/Desktop/projetos-2025/ICMBIO/project/exemplo/Ocorrencia.kml"

In [3]:
gdf = parse_kml(file_name)

2025-12-04 16:48:28.055 | INFO     | src.copypreprocess:parse_kml:45 - Starting KML parsing for file: /media/manecao/HD/Desktop/projetos-2025/ICMBIO/project/exemplo/Ocorrencia.kml
2025-12-04 16:48:28.079 | DEBUG    | src.copypreprocess:parse_kml:51 - KML file parsed successfully
2025-12-04 16:48:28.109 | ERROR    | src.copypreprocess:parse_kml:91 - Failed to parse KML file: Assigning CRS to a GeoDataFrame without a geometry column is not supported. Supply geometry using the 'geometry=' keyword argument, or by providing a DataFrame with column name 'geometry'


ValueError: Assigning CRS to a GeoDataFrame without a geometry column is not supported. Supply geometry using the 'geometry=' keyword argument, or by providing a DataFrame with column name 'geometry'